## CONFIG

In [1]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd
from bs4 import BeautifulSoup



from collections import Counter

from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [2]:
codice_galattico = "../Hackapizza Dataset/Codice Galattico/Codice Galattico.pdf"
manuale_cucina = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"

domande = "../Hackapizza Dataset/domande.csv"
piatti = "../Hackapizza Dataset/Misc/dish_mapping.json"
pianeti = "../Hackapizza Dataset/Misc/Distanze.csv"

blog_1 = "../Hackapizza Dataset/Blogpost/blog_etere_del_gusto.html"
blog_2 = "../Hackapizza Dataset/Blogpost/blog_sapore_del_dune.html"


load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [3]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [4]:
def estrai_testo_pdf(percorso_pdf):
    with open(percorso_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        testo = ""
        for pagina in reader.pages:
            testo += pagina.extract_text()
    return testo

def estrai_testo_html(percorso_html):
    """Estrae il testo da un file HTML."""
    with open(percorso_html, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        return soup.get_text(separator=" ", strip=True)

## METODI

In [7]:
from json_repair import repair_json

def estrai_valorizzazioni(testo, categoria, llm, driver):
    """
    Estrae le valorizzazioni di entità di una specifica categoria dal testo.
    """
    # Genera un prompt basato sulle entità filtrate     
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti, Tecniche e Licenze.
    
    Il testo che segue potrebbe contenere informazioni pertinenti a questa categoria: {categoria}
    In output voglio tutte le possibile istanze di questa categoria, come entità da caricare su neo4j.
    Per ogni istanza recupera:
        - NOME dell'istanza
        - eventuali PARAMETRI associati all'istanza. Il singolo PARAMETRO deve essere una STRINGA.
    Attento che i nomi delle chiavi devono essere identiche a quelle fornite.
    
    Restituisci SOLO il JSON valido e completo, senza formattazione markdown o decoratori.

    Il formato deve essere:
    [
      {{'NOME': 'nome', "PARAMETRI": {{...parametri...}}}},   
    ]  
    
    Testo:
    {testo}
    """
    
    # Chiamata al modello LLM per ottenere la valorizzazione delle entità
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('```json'):
        content = content[7:].strip()
    elif content.startswith('json'):
        content = content[4:].strip()
    
    # Tentativo di riparare il JSON
    try:
        valorizzazioni = json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing JSON per entità {categoria}: {str(e)}")
        try:
            valorizzazioni = json.loads(repair_json(content))
        except Exception:
            # Fallback con richiesta all'LLM
            prompt_repair = f"Correggi questo JSON per Neo4j e assicurati che sia valido e completo:\n{content}"
            repaired = llm.invoke(prompt_repair).content.strip()
            
            # Pulizia del JSON riparato
            if repaired.startswith('```') and repaired.endswith('```'):
                repaired = '\n'.join(repaired.split('\n')[1:-1])
            if repaired.startswith('```json'):
                repaired = repaired[7:].strip()
            elif repaired.startswith('json'):
                repaired = repaired[4:].strip()
                
            try:
                valorizzazioni = json.loads(repaired)
            except:
                print(f"Impossibile riparare il JSON per le entità di tipo {categoria}")
                valorizzazioni = []
    
    # Assicurati che il risultato sia una lista
    if not isinstance(valorizzazioni, list):
        valorizzazioni = [valorizzazioni]
    
    return valorizzazioni

In [9]:
import json
from json_repair import repair_json

def estrai_relazioni(testo, relazioni_info, llm, driver):
    """
    Estrae entità e relazioni tra di esse da un testo.
    
    Args:
        testo: Il testo da cui estrarre le relazioni
        relazioni_info: Dizionario nella forma {'NOME_RELAZIONE': ['Tipo_Entità1', 'Tipo_Entità2']}
        llm: Modello di linguaggio da utilizzare
        driver: Driver Neo4j
        
    Returns:
        Dizionario contenente entità e relazioni estratte
    """
    risultati = {}
    entita_tipi = set()
    
    # Raccogli tutti i tipi di entità unici dalle relazioni
    for _, [tipo_entita1, tipo_entita2] in relazioni_info.items():
        entita_tipi.add(tipo_entita1)
        entita_tipi.add(tipo_entita2)
    
    # Estrai tutte le entità una sola volta per tipo
    for tipo_entita in entita_tipi:
        print(f"Estraendo entità di tipo '{tipo_entita}'...")
        entita = estrai_valorizzazioni(testo, tipo_entita, llm, driver)
        risultati[tipo_entita] = entita
    
    # Ora estrai le relazioni
    for nome_relazione, [tipo_entita1, tipo_entita2] in relazioni_info.items():
        print(f"Estraendo relazioni '{nome_relazione}' tra '{tipo_entita1}' e '{tipo_entita2}'...")
        
        # Estrai le relazioni tra entità1 e entità2
        relazioni = estrai_valorizzazioni_relazioni(
            testo, 
            nome_relazione, 
            tipo_entita1, 
            tipo_entita2, 
            risultati[tipo_entita1], 
            risultati[tipo_entita2], 
            llm, 
            driver
        )
        
        # Salva le relazioni nel risultato
        risultati[nome_relazione] = relazioni
        
    return risultati

In [8]:
def estrai_valorizzazioni_relazioni(testo, nome_relazione, tipo_entita1, tipo_entita2, entita1, entita2, llm, driver):
    """
    Estrae le relazioni specifiche tra entità di due tipi.
    """
    # Genera un prompt per estrarre le relazioni
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti, Tecniche e Licenze.
    
    Il testo che segue potrebbe contenere informazioni su relazioni di tipo '{nome_relazione}' tra entità di tipo '{tipo_entita1}' e '{tipo_entita2}'.
    
    In output voglio tutte le relazioni di tipo '{nome_relazione}' come triple da caricare su neo4j.
    Per ogni relazione recupera:
        - ORIGINE: nome dell'entità di tipo '{tipo_entita1}'
        - DESTINAZIONE: nome dell'entità di tipo '{tipo_entita2}'
        - eventuali PARAMETRI associati alla relazione
    
    Ecco le entità di tipo '{tipo_entita1}' già identificate:
    {[e.get("NOME", "") for e in (entita1 if isinstance(entita1, list) else [entita1])]}
    
    Ecco le entità di tipo '{tipo_entita2}' già identificate:
    {[e.get("NOME", "") for e in (entita2 if isinstance(entita2, list) else [entita2])]}
    
    Restituisci SOLO il JSON valido e completo, senza formattazione markdown o decoratori.
    Il formato deve essere:
    [
      {{"ORIGINE": "nome_entita1", "DESTINAZIONE": "nome_entita2", "PARAMETRI": {{...parametri...}} }}
    ]
    
    
    Testo:
    {testo}
    """
    
    # Chiamata al modello LLM per ottenere la valorizzazione delle relazioni
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('```json'):
        content = content[7:].strip()
    elif content.startswith('json'):
        content = content[4:].strip()
    
    # Tentativo di riparare il JSON
    try:
        relazioni = json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing JSON delle relazioni {nome_relazione}: {str(e)}")
        try:
            relazioni = json.loads(repair_json(content))
        except Exception:
            # Fallback con richiesta all'LLM
            prompt_repair = f"Correggi questo JSON per Neo4j e assicurati che sia valido e completo:\n{content}"
            repaired = llm.invoke(prompt_repair).content.strip()
            
            # Pulizia del JSON riparato
            if repaired.startswith('```') and repaired.endswith('```'):
                repaired = '\n'.join(repaired.split('\n')[1:-1])
            if repaired.startswith('```json'):
                repaired = repaired[7:].strip()
            elif repaired.startswith('json'):
                repaired = repaired[4:].strip()
                
            try:
                relazioni = json.loads(repaired)
            except:
                print(f"Impossibile riparare il JSON delle relazioni {nome_relazione}")
                relazioni = []
    
    # Assicurati che il risultato sia una lista
    if not isinstance(relazioni, list):
        relazioni = [relazioni]
    
    return relazioni

In [10]:
def carica_entita_e_relazioni_su_neo4j(risultati, relazioni_info, driver):
    """
    Carica entità e relazioni su Neo4j gestendo strutture dati nidificate
    
    Args:
        risultati: Dizionario contenente entità e relazioni
        relazioni_info: Dizionario delle relazioni come riferimento
        driver: Driver Neo4j
    """
    # Crea un set di tutti i tipi di relazione
    tipi_relazione = set(relazioni_info.keys())
    
    # Contatori per verifica
    entita_create = 0
    relazioni_create = 0
    
    # Verifica preliminare che il driver sia connesso
    try:
        with driver.session() as session:
            result = session.run("RETURN 1 AS test")
            test_value = result.single()["test"]
            if test_value != 1:
                print("ERRORE: Connessione al database non funzionante")
                return
            print("Connessione al database verificata con successo")
    except Exception as e:
        print(f"ERRORE: Impossibile connettersi al database: {str(e)}")
        return
    
    # Carica le entità
    for tipo_entita, entita_data in risultati.items():
        # Salta le chiavi che sono nomi di relazione
        if tipo_entita in tipi_relazione:
            continue
        
        print(f"Elaborazione per tipo: {tipo_entita}")
        print(f"Tipo di dato: {type(entita_data).__name__}")
        
        # Gestione speciale per struttura dati nidificata
        if isinstance(entita_data, dict) and tipo_entita in entita_data:
            print(f"Rilevata struttura nidificata per {tipo_entita}")
            if isinstance(entita_data[tipo_entita], list):
                entita_list = entita_data[tipo_entita]
            else:
                entita_list = [entita_data[tipo_entita]]
            print(f"Estratti {len(entita_list)} elementi dalla struttura nidificata")
        elif isinstance(entita_data, list):
            entita_list = entita_data
        elif isinstance(entita_data, dict) and "NOME" in entita_data:  # caso di singolo oggetto
            entita_list = [entita_data]
        else:
            print(f"ERRORE: Formato non supportato per {tipo_entita}: {entita_data}")
            continue
        
        print(f"Caricando {len(entita_list)} entità di tipo {tipo_entita}...")
        
        for entita in entita_list:
            nome = entita.get("NOME", "")
            if not nome:  
                print(f"ERRORE: Entità di tipo {tipo_entita} senza nome: {entita}")
                continue
                
            parametri = entita.get("PARAMETRI", {})
            
            print(f"Creazione entità: {nome} (tipo: {tipo_entita})")
            
            # Usa una transazione esplicita
            with driver.session() as session:
                tx = session.begin_transaction()
                try:
                    # Crea la query base
                    query = f"MERGE (e:{tipo_entita} {{nome: $nome}})"
                    
                    # Se ci sono parametri, aggiungi SET
                    if parametri:
                        query += " SET "
                        set_clauses = []
                        for k in parametri.keys():
                            # Gestione speciale per parametri che sono dizionari
                            if isinstance(parametri[k], dict):
                                # Converti sotto-dizionari in formato JSON o string
                                set_clauses.append(f"e.{k} = $param_{k}")
                            else:
                                set_clauses.append(f"e.{k} = ${k}")
                        query += ", ".join(set_clauses)
                    
                    query += " RETURN e"  # Ritorna il nodo per conferma
                    
                    # Prepara i parametri per la query
                    params = {"nome": nome}
                    for k, v in parametri.items():
                        if isinstance(v, dict):
                            params[f"param_{k}"] = v  # Neo4j supporta JSON per le proprietà
                        else:
                            params[k] = v
                    
                    print(f"Esecuzione query: {query}")
                    result = tx.run(query, **params)
                    record = result.single()
                    if record:
                        node = record.get("e")
                        print(f"Entità creata e confermata: {nome} di tipo {tipo_entita}")
                        entita_create += 1
                    else:
                        print(f"ATTENZIONE: Entità {nome} non confermata dopo l'esecuzione della query")
                    
                    # Esplicita conferma della transazione
                    tx.commit()
                except Exception as e:
                    tx.rollback()
                    print(f"ERRORE nel caricamento dell'entità {nome}: {str(e)}")
                    print(f"Query: {query}")
                    print(f"Parametri: {params}")
    
    # Carica le relazioni
    for tipo_relazione, relazioni in risultati.items():
        # Salta le chiavi che non sono tipi di relazione
        if tipo_relazione not in tipi_relazione:
            continue
            
        # Determina i tipi di entità coinvolti nella relazione
        tipo_origine, tipo_destinazione = relazioni_info[tipo_relazione]
        
        print(f"Caricando {len(relazioni)} relazioni di tipo {tipo_relazione}...")
        
        for relazione in relazioni:
            origine = relazione.get("ORIGINE", "")
            destinazione = relazione.get("DESTINAZIONE", "")
            
            if not origine or not destinazione:
                print(f"ERRORE: Relazione {tipo_relazione} incompleta: {relazione}")
                continue
                
            parametri = relazione.get("PARAMETRI", {})
            
            # Usa una transazione esplicita
            with driver.session() as session:
                # Prima verifica che i nodi esistano
                verify_query = f"""
                OPTIONAL MATCH (o:{tipo_origine} {{nome: $origine}})
                OPTIONAL MATCH (d:{tipo_destinazione} {{nome: $destinazione}})
                RETURN o, d
                """
                verify_result = session.run(verify_query, origine=origine, destinazione=destinazione)
                record = verify_result.single()
                
                origin_node = record.get("o")
                dest_node = record.get("d")
                
                if not origin_node:
                    print(f"ERRORE: Nodo origine '{origine}' di tipo {tipo_origine} non trovato")
                    print(f"Creazione automatica del nodo origine...")
                    try:
                        session.run(f"CREATE (n:{tipo_origine} {{nome: $nome}})", nome=origine)
                        print(f"Nodo origine '{origine}' creato")
                    except Exception as e:
                        print(f"ERRORE nella creazione automatica: {str(e)}")
                        continue
                
                if not dest_node:
                    print(f"ERRORE: Nodo destinazione '{destinazione}' di tipo {tipo_destinazione} non trovato")
                    print(f"Creazione automatica del nodo destinazione...")
                    try:
                        session.run(f"CREATE (n:{tipo_destinazione} {{nome: $nome}})", nome=destinazione)
                        print(f"Nodo destinazione '{destinazione}' creato")
                    except Exception as e:
                        print(f"ERRORE nella creazione automatica: {str(e)}")
                        continue
                
                # Crea la relazione
                tx = session.begin_transaction()
                try:
                    # Crea la query per la relazione
                    query = f"""
                    MATCH (o:{tipo_origine} {{nome: $origine}})
                    MATCH (d:{tipo_destinazione} {{nome: $destinazione}})
                    MERGE (o)-[r:{tipo_relazione}]->(d)
                    """
                    
                    # Se ci sono parametri, aggiungi SET
                    if parametri:
                        query += " SET "
                        set_clauses = []
                        for k in parametri.keys():
                            if isinstance(parametri[k], dict):
                                set_clauses.append(f"r.{k} = $param_{k}")
                            else:
                                set_clauses.append(f"r.{k} = ${k}")
                        query += ", ".join(set_clauses)
                    
                    query += " RETURN r"  # Ritorna la relazione per conferma
                    
                    # Prepara i parametri per la query
                    params = {
                        "origine": origine,
                        "destinazione": destinazione
                    }
                    for k, v in parametri.items():
                        if isinstance(v, dict):
                            params[f"param_{k}"] = v
                        else:
                            params[k] = v
                    
                    result = tx.run(query, **params)
                    rel = result.single()
                    if rel:
                        print(f"Relazione creata e confermata: {origine}-[{tipo_relazione}]->{destinazione}")
                        relazioni_create += 1
                    else:
                        print(f"ATTENZIONE: Relazione {origine}-[{tipo_relazione}]->{destinazione} non confermata")
                    
                    # Esplicita conferma della transazione
                    tx.commit()
                except Exception as e:
                    tx.rollback()
                    print(f"ERRORE nel caricamento della relazione {origine}-[{tipo_relazione}]->{destinazione}: {str(e)}")
                    print(f"Query: {query}")
                    print(f"Parametri: {params}")
    
    print(f"\nRiepilogo: create {entita_create} entità e {relazioni_create} relazioni")
    
    # Verifica finale: query per contare entità e relazioni
    try:
        with driver.session() as session:
            # Conta tutte le entità
            count_query = "MATCH (n) RETURN count(n) as count"
            count_result = session.run(count_query)
            node_count = count_result.single()["count"]
            
            # Conta tutte le relazioni
            rel_query = "MATCH ()-[r]->() RETURN count(r) as count"
            rel_result = session.run(rel_query)
            rel_count = rel_result.single()["count"]
            
            print(f"Verifica finale dal database: {node_count} nodi e {rel_count} relazioni presenti")
            
            # Elenca i primi 5 nodi per ogni tipo
            print("\nEsempi di nodi per tipo:")
            type_query = "CALL db.labels() YIELD label RETURN label"
            labels_result = session.run(type_query)
            for label_record in labels_result:
                label = label_record["label"]
                nodes_query = f"MATCH (n:{label}) RETURN n.nome as nome LIMIT 5"
                nodes_result = session.run(nodes_query)
                nodes = [record["nome"] for record in nodes_result]
                print(f"  {label}: {', '.join(nodes) if nodes else 'Nessun nodo trovato'}")
    except Exception as e:
        print(f"ERRORE nella verifica finale: {str(e)}")

In [88]:
import re

def chunk_text(text, max_chars=3000, overlap=300):
    """
    Divide un testo in chunk di massimo `max_chars` caratteri, con `overlap` tra i chunk.
    
    Args:
        text (str): Il testo da spezzare in chunk.
        max_chars (int): Numero massimo di caratteri per chunk.
        overlap (int): Numero di caratteri di sovrapposizione tra i chunk.
    
    Returns:
        list: Lista di chunk di testo.
    """
    # Pulizia del testo per evitare problemi di spazi extra
    text = re.sub(r'\n+', '\n', text.strip())

    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap  # Sposta l'inizio del prossimo chunk per avere overlap
    
    return chunks

In [ ]:
chunks = chunk_text(testo_menu, 3000, 400)


In [94]:
len(chunks)

9

## MAIN

In [11]:
# Esegui l'analisi del PDF
testo_manuale = estrai_testo_pdf(manuale_cucina)
# Esegui l'analisi del PDF
testo_codice = estrai_testo_pdf(codice_galattico)

In [14]:
capitoli_manuale = ["Cap" + t for t in testo_manuale.split("Cap")]
capitoli_manuale[0] = capitoli_manuale[0][3:]
capitoli_manuale

['Introduzione\nIo sono il grande Sirius Cosmo, lo chef stellare per eccellenza. Se non mi conoscete ancora, preparatevi:\nil mio nome è sinonimo di cucina galattica. In questo manuale vi insegnerò tutto quello che serve per\ndiventare veri cuochi, da Alpha Centauri fino alla Nebulosa del Granchio (dove, a proposito, non hanno\nneanche un crostaceo decente). Impareremo insieme le licenze e le abilità fondamentali per cucinare\nnello spazio senza mandare in tilt lʼintero sistema di supporto vitale della nave — perché diciamocelo, un\nbuon brasato non vale una depressurizzazione dʼemergenza.\nVi guiderò passo passo tra tecniche di cucina gravitazionali, segreti per friggere senza far esplodere la\ncabina e trucchi per ottenere il perfetto soufflé orbitale.\nSpoiler: Sì, si può montare una maionese in assenza di gravità — basta avere il giusto polso e un poʼ di\npazienza, cose che non si insegnano nei manuali tecnici ma che io, Sirius Cosmo, sono qui per\ntrasmettervi.\nPronti a decollare

In [ ]:
import re

def split_string(input_string):    
    #pattern = r'(?<=\s|\n)\d(?=\s)' Nope
    #pattern = r'(?<="  ")\d(?=\s)' Nope
    pattern = r'(?<=  )\d(?= )' # Ding Ding !!
    return re.split(pattern, input_string)

split_string(testo_codice)

['  Gazzetta u(iciale del Consiglio Intergalattico   789/12773  DECRETO SUPREMO 789/12773 DEL GRAN CONSIGLIO  del 67° Giorno del Ciclo Cosmico 789  sulla regolamentazione della sicurezza alimentare intergalattica    IL GRAN CONSIGLIO DELLA FEDERAZIONE INTERGALATTICA,  Nel rispetto delle diverse credenze e tradizioni alimentari,  Viste le necessità ﬁsiologiche di tutte le specie senzienti e non senzienti come deﬁnite dal regolamento 675/12,   Con particolare attenzione agli individui iscritti ai gruppi:  (1) I membri dell’Ordine della Galassia di Andromeda, deﬁniti nella sezione 1.1  (2) I membri dell’Ordine dei Naturalisti, le cui rigide prescrizioni alimentari vengono discusse nella sezione 1.2  (3) Gli individui all’Ordine degli Armonisti, la cui ﬁlosoﬁa è meglio indicata nella sezione 1.3  STABILISCE IL PRESENTE CODICE COME FONDAMENTO NORMATIVO PER L\'ESERCIZIO DELLE ATTIVITÀ DI RISTORAZIONE NEL TERRITORIO DELLA FEDERAZIONE.     \n789/12773     67° Giorno del Ciclo Cosmico 789 \n1 D

In [27]:
estrai_relazioni(capitoli_manuale[0], relazioni, llm, driver)

Estraendo entità di tipo 'Licenza'...
Estraendo entità di tipo 'Ristorante'...
Estraendo entità di tipo 'Tecnica'...
Estraendo entità di tipo 'Ingrediente'...
Estraendo entità di tipo 'Piatto'...
Estraendo entità di tipo 'Pianeta'...
Estraendo relazioni 'CONTIENE' tra 'Piatto' e 'Ingrediente'...
Estraendo relazioni 'PREPARATO' tra 'Piatto' e 'Tecnica'...
Estraendo relazioni 'OFFERTA' tra 'Ristorante' e 'Piatto'...
Estraendo relazioni 'SITUATO' tra 'Ristorante' e 'Pianeta'...
Estraendo relazioni 'HA' tra 'Ristorante' e 'Licenza'...
Errore nel parsing JSON delle relazioni HA: Extra data: line 2 column 1 (char 3)


{'Licenza': [],
 'Ristorante': [],
 'Tecnica': [{'NOME': 'Cucina gravitazionale',
   'PARAMETRI': {'tipo': 'gravitazionale'}},
  {'NOME': 'Friggere senza far esplodere la cabina',
   'PARAMETRI': {'contesto': 'evitare esplosione cabina'}},
  {'NOME': 'Soufflé orbitale',
   'PARAMETRI': {'condizione': 'orbitale', 'qualità': 'perfetto'}},
  {'NOME': 'Montare maionese in assenza di gravità',
   'PARAMETRI': {'tipo': 'maionese', 'condizione': 'assenza di gravità'}}],
 'Ingrediente': [{'NOME': 'maionese', 'PARAMETRI': {}}],
 'Piatto': [{'NOME': 'brasato',
   'PARAMETRI': {'descrizione': "buono, non vale una depressurizzazione d'emergenza"}},
  {'NOME': 'soufflé orbitale', 'PARAMETRI': {'descrizione': 'perfetto'}},
  {'NOME': 'maionese',
   'PARAMETRI': {'condizione': 'in assenza di gravità',
    'note': "basta avere il giusto polso e un po' di pazienza"}}],
 'Pianeta': [],
 'CONTIENE': [{'ORIGINE': 'maionese',
   'DESTINAZIONE': 'maionese',
   'PARAMETRI': {}}],
 'PREPARATO': [{'ORIGINE': '

In [25]:
# Carica il file CSV
df_domande = pd.read_csv(domande)
testo_domande = df_domande.to_string()  # Converti il CSV in testo

# Carica il file CSV
df_pianeti = pd.read_csv(pianeti)
testo_pianeti = df_pianeti.to_string()  # Converti il CSV in testo

# Esempio di utilizzo:
testo_blog1 = estrai_testo_html(blog_1)
# Esempio di utilizzo:
testo_blog2 = estrai_testo_html(blog_2)

In [26]:
relazioni = {'CONTIENE': ['Piatto', 'Ingrediente'], 'PREPARATO': ['Piatto', 'Tecnica'], 'OFFERTA': ['Ristorante', 'Piatto'], 'SITUATO': ['Ristorante', 'Pianeta'], 'HA': ['Ristorante', 'Licenza']}

In [ ]:
# Estrai entità e relazioni
chunks = chunk_text(testo_manuale, 3000, 400)
for chunk in chunks:
    risultati = estrai_relazioni(chunk, relazioni, llm, driver)
    carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)

In [ ]:
# Estrai entità e relazioni
chunks = chunk_text(testo_codice, 3000, 400)
for chunk in chunks:
    risultati = estrai_relazioni(chunk, relazioni, llm, driver)
    carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)

In [ ]:
# Estrai entità e relazioni
chunks = chunk_text(testo_blog1, 3000, 400)
for chunk in chunks:
    risultati = estrai_relazioni(chunk, relazioni, llm, driver)
    carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)

In [ ]:
# Estrai entità e relazioni
chunks = chunk_text(testo_blog2, 3000, 400)
for chunk in chunks:
    risultati = estrai_relazioni(chunk, relazioni, llm, driver)
    carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)

In [ ]:
import os
from pathlib import Path

# Percorso della cartella contenente i file PDF dei menu
menu_folder = "../Hackapizza Dataset/Menu/"

# Ottieni la lista di tutti i file PDF nella cartella
pdf_files = [f for f in os.listdir(menu_folder) if f.endswith('.pdf')]

# Per ogni file PDF nella cartella
for pdf_file in pdf_files:
    try:
        # Costruisci il percorso completo del file
        pdf_path = os.path.join(menu_folder, pdf_file)
        
        print(f"Elaborazione del file: {pdf_file}")
        
        # Estrai il testo dal PDF
        testo_menu = estrai_testo_pdf(pdf_path)
        
        # Estrai entità e relazioni
        chunks = chunk_text(testo_menu, 3000, 400)
        for chunk in chunks:
            risultati = estrai_relazioni(chunk, relazioni, llm, driver)
            carica_entita_e_relazioni_su_neo4j(risultati, relazioni, driver)
        
        print(f"Elaborazione di {pdf_file} completata con successo")
        print("-" * 50)
        
    except Exception as e:
        print(f"Errore durante l'elaborazione di {pdf_file}: {str(e)}")
        # Continua con il prossimo file anche in caso di errore
        continue

Elaborazione del file: Anima Cosmica.pdf
  Estrazione Tecnica da Anima Cosmica.pdf
  Caricamento di 19 entità di tipo Tecnica
Caricata entità Cottura a Vapore in categoria Tecnica
Caricata entità Cottura a Vapore in categoria Tecnica
Caricata entità Taglio a Risonanza Sonica in categoria Tecnica
Caricata entità Cottura in categoria Tecnica
Caricata entità Affumicatura in categoria Tecnica
Caricata entità Cottura a Forno in categoria Tecnica
Caricata entità Saltare in Padella in categoria Tecnica
Caricata entità Marinatura in categoria Tecnica
Caricata entità Congelamento in categoria Tecnica
Caricata entità Sinergia Elettro-Osmotica in categoria Tecnica
Caricata entità Bollitura Infrasonica in categoria Tecnica
Caricata entità Marinatura in categoria Tecnica
Caricata entità Grigliatura Eletro-Molecolare in categoria Tecnica
Caricata entità Cottura a Vapore in categoria Tecnica
Caricata entità Cottura Idrodinamica in categoria Tecnica
Caricata entità Bollitura Termografica in categoria 